In [1]:
from lifelines import KaplanMeierFitter
from lifelines.utils import to_long_format
from lifelines.utils import add_covariate_to_timeline
import pandas as pd
import numpy as np
import pickle
from itertools import repeat

In [2]:
df = pd.read_csv('Imaging_clinical_feature_set_folds_outcomes.csv')

In [3]:
df_rel = df.fillna('N/A')

In [4]:
## drop the first visit

In [5]:
df_rel = df[df['Max. months remain dry']!='N/A']
df_rel_dropfirst = df_rel[df_rel['Elapsed time since first imaging']!=0.000000]

c:\users\imonb\appdata\local\programs\python\python35\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
patient_ids = df_rel['Patient number'].unique()

In [7]:
## Get first enrty for each patient

In [8]:
Time_D = []
Event_D = []
Visit = []

for i in range(len(patient_ids)):
    temp = df_rel[df_rel['Patient number']==patient_ids[i]]
    test = temp.sort_values('Elapsed time since first imaging', ascending=True)
    T = 0
    E = 0
    flg = 0
    for j in range(test.shape[0]):
        if test.iloc[j]['Progression during study'] == 1 and flg ==0:
            T = test.iloc[j]['Elapsed time since first imaging']
            E = 1
            flg = 1
        else:
            T = test.iloc[j]['Elapsed time since first imaging']
    Time_D.append(T)
    Event_D.append(E)
    Visit.append(test.shape[0])

In [9]:
df_H = pd.DataFrame({'Time':Time_D, 'Event':Event_D, 'Visit':Visit})

In [10]:
## Regression dataset

In [11]:
df_regression = df_rel.loc[:,'Age':'Smoking: Previous cigarette smoker']

In [12]:
df_regression['Elapsed time since first imaging'] = df_rel['Elapsed time since first imaging']

In [13]:
df_regression = df_regression.drop('Race: Multiracial',1)
df_regression['Patient number'] = df_rel['Patient number']
df_regression = df_regression.drop_duplicates('Patient number')
df_regression['Duration'] = Time_D
df_regression['Event'] = Event_D

In [14]:
race = list(repeat(0,df_regression.shape[0]))
for i in range(df_regression.shape[0]):
    if df_regression.iloc[i]['Race: White'] == 1:
        race[i] = 1
    if df_regression.iloc[i]['Race: Native Hawaiian or Other Pacific Islander'] == 1:
        race[i] = 2
    if df_regression.iloc[i]['Race: Black or African American'] == 1:
        race[i] = 3
    if df_regression.iloc[i]['Race: Asian'] == 1:
        race[i] = 4
    if df_regression.iloc[i]['Race: American Indian or Alaska Native'] == 1:
        race[i] = 5

In [15]:
df_regression['Race'] = race
df_regression = df_regression.drop('Race: White',1)
df_regression = df_regression.drop('Race: Native Hawaiian or Other Pacific Islander',1)
df_regression = df_regression.drop('Race: Black or African American',1)
df_regression = df_regression.drop('Race: Asian',1)
df_regression = df_regression.drop('Race: American Indian or Alaska Native',1)
df_regression = df_regression.drop('Patient number',1)

In [16]:
duration = []
event = []
for i in range(len(patient_ids)):
    temp = df_rel[df_rel['Patient number']==patient_ids[i]]
    E = list(temp['Elapsed time since first imaging'])
    duration.append(max(E))
    E = list(temp['Progression during study'])
    if 1 in E:
        event.append(1)
    else:
        event.append(0)

In [17]:
base_df =  df_regression

In [18]:
base_df['Patient number'] = patient_ids
df_future = df_rel.loc[:, 'Numberofdrusenincube':'Cirrusdrusenvolume5mm']
df_future = df_future.dropna(axis=1, how='any')

In [19]:
from sklearn import preprocessing

x = df_future.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_future = pd.DataFrame(x_scaled, columns = list(df_rel.loc[:, 'Numberofdrusenincube':'Cirrusdrusenvolume5mm']))

In [20]:
df_future['Patient number'] = df_rel['Patient number']
df_future['Elapsed time since first imaging'] = df_rel['Elapsed time since first imaging']

In [21]:
df_future['Patient number'] = df_rel['Patient number']
df_future['Age'] = df_rel['Age']

In [22]:
base_df = to_long_format(base_df, duration_col="Duration")
df_cov = add_covariate_to_timeline(base_df, df_future, duration_col='Elapsed time since first imaging', id_col='Patient number', event_col='Event')

In [23]:
df_cov = df_cov.dropna(axis=0, how='any')
df_cov.to_csv('Modified_data.csv')